#  Tugas Akhir BTPN Syariah Data Engineer Virtual Internship Program


Project tugas akhir ini dibuat guna mengevaluasi pembelajaran pada BTPN Syariah Data Engineer Virtual Internship Program Rakamin khususnya pada pemahaman SQL dan Tableau.

### Permasalahan
Seorang manajer di bank merasa terganggu dengan semakin banyak pelanggan yang meninggalkan layanan kartu kredit mereka. Mereka akan sangat menghargai jika seseorang dapat mengetahui profil pelanggan sehingga mereka dapat mengetahui pelanggan mana yang akan pergi sehingga mereka dapat secara proaktif mendatangi pelanggan untuk memberikan layanan yang lebih baik dan mengubah
keputusan pelanggan ke arah yang berlawanan.

### Import library 
Pada project ini menggunakan library Pandas, Glob, dan Sqlalchemy

In [1]:
#import library yang dibutuhkan
import pandas as pd
import glob
from sqlalchemy import create_engine

In [2]:
#inisiasi koneksi ke database postgresql
user = 'postgres'
password = '0078'
host = 'localhost'
port = 5432
db_name = 'vie_btpns'

In [3]:
#konek ke potgresql
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db_name}")
engine.connect()

### Load data csv ke database
Pada project ini terdapat 5 data csv yang akan diimport ke PostgreSQL dengan keterangan: <br>
Customer_data_history = data historical customer sampai saat ini<br>
Keterangan kolom:<br>
    -    Clientnum                   : nomor id client <br>
    -    Idstatus                    : keterangan status customer<br>
    -    Customer_age                : umur customer<br>
    -    Gender                      : jenis kelamin customer<br>
    -    Dependent_count             : yang menjadi tanggungan customer<br>
    -    Educationid                 : keterangan jenjang Pendidikan customer<br>
    -    Maritalid                   : status pernikahan customer<br>
    -    Income_category             : ketgori penghasilan customer<br>
    -    Card_categoryid             : jenis kartu kredit customer<br>
    -    Month_on_book               : periode berhubungan dengan bank<br>
    -    Relationship_in_count       : total product yang dipegang customer<br>
    -    Months_inactive_in_12_month : jumlah bulan tidak aktif selama 12 bulan terakhir<br>
    -    Contacts_Count_12_mon       : total dihubungi bank dalam 12 bulan terakhir<br>
    -    Credit_limit                : limit credit<br>
    -    Total_Revolving_Balance_on_the_Credit_Card : total saldo bergulir pada kartu kredit<br>
    -    Avg_open_to_buy             : membeli dengan kartu kredit dalam 12 bulan terakhir<br>
    -    Total_trans_amt             : jumlah transaksi<br>
    -    Total_trans_ct              : frekuensi transaksi<br>
    -    Avg_utilization_ratio       : rata rata rasio penggunaan kartu kredit<br>

Category_db = data category dari layanan kartu kredit yang digunakan<br>
    -    id            : id kategori<br>
    -    Card_category : jenis kartu kredit customer<br>
        
Education_db = data tingkat Pendidikan customer<br>
    -    id            : id level edukasi<br>
    -    Education_Level : keterangan jenjang Pendidikan customer<br>

Marital_db = data status pernikahan customer<br>
    -    id            : id marital<br>
    -    Marital_Status : status pernikahan customer<br>

Status_db = data keterangan status customer existing / attired<br>
    -    id            : id status<br>
    -    status        : keterangan status customer<br>

In [4]:
#read file csv
#creat
path = "dataset"
csv_files = glob.glob(path + "/*.csv")
for f in csv_files:
    df = pd.read_csv(f)
    df.columns= df.columns.str.lower()
    table_name = f.split("\\")[-1].split(".")[0]
  
    #import csv ke postgresql
    # print(pd.io.sql.get_schema(df, name=table_name, con=engine))
    df.to_sql(name= table_name, con=engine, if_exists="replace", index=False)
    print('Ingesting ', f.split("\\")[-1], ' file')

Ingesting  category_db.csv  file
Ingesting  customer_data_history.csv  file
Ingesting  education_db.csv  file
Ingesting  marital_db.csv  file
Ingesting  status_db.csv  file


berikut adalah desain ERD pada projek ini
<br><br>
<img src='erd.png' height=500px>

In [6]:
query = """
SELECT tablename
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
"""

pd.read_sql(query, con=engine)

,tablename
0,education_db
1,marital_db
2,category_db
3,customer_data_history
4,status_db


### Eksplorasi Data
Pada project ini eksplorasi data menggunakan query SQL 
1. Melakukan join data 
2. Mengelompokan customer berdasarkan umur
3. Mengelompokan customer berdasarkan interval gaji

In [11]:
query2 = """ 
    SELECT 
        a.clientnum, a.customer_age, a.dependent_count, a.gender,
        a.income_category, a.months_on_book, 
        a.total_relationship_count,a.months_inactive_12_mon,a.credit_limit,a.total_revolving_bal,
        a.total_trans_amt, a.avg_open_to_buy, a.total_trans_ct, a.avg_utilization_ratio,
        b.status , c.card_category, d.education_Level, e.marital_status 
    FROM
        customer_data_history AS a
    LEFT JOIN status_db AS b on a.idstatus = b.id
    LEFT JOIN category_db AS c on a.card_categoryid = c.id
    LEFT JOIN education_db AS d on a.educationid = d.id
    LEFT JOIN marital_db AS e on a.maritalid = e.id;
"""
customer_data =pd.read_sql(query2, con=engine)
customer_data.to_sql(name= 'customer_data', con=engine, if_exists="replace", index=False)
customer_data.head()


,clientnum,customer_age,dependent_count,gender,income_category,months_on_book,total_relationship_count,months_inactive_12_mon,credit_limit,total_revolving_bal,total_trans_amt,avg_open_to_buy,total_trans_ct,avg_utilization_ratio,status,card_category,education_level,marital_status
0,716474358,43,3,F,Less than $40K,35,5,3,1721.0,729,4853,992.0,74,0.424,Existing Customer,Blue,Unknown,Married
1,818770008,49,5,F,Less than $40K,44,6,1,8256.0,864,1291,7392.0,33,0.105,Existing Customer,Blue,Graduate,Single
2,713982108,51,3,M,$80K - $120K,36,4,1,3418.0,0,1887,3418.0,20,0.000,Existing Customer,Blue,Graduate,Married
3,769911858,40,4,F,Less than $40K,34,3,4,3313.0,2517,1171,796.0,20,0.760,Existing Customer,Blue,High School,Unknown
4,709106358,40,3,M,$60K - $80K,21,5,1,4716.0,0,816,4716.0,28,0.000,Existing Customer,Blue,Uneducated,Married


In [12]:
query3 = """ 
SELECT 
CASE WHEN customer_age <20 THEN '0-20' 
WHEN customer_age BETWEEN 20 AND 30 THEN '20-30'
WHEN customer_age BETWEEN 30 AND 40 THEN '30-40'
WHEN customer_age BETWEEN 40 AND 50 THEN '40-50'
WHEN customer_age BETWEEN 50 AND 60 THEN '50-60' ELSE '60++' 
END AS age_range,

COUNT (case when gender = 'M' then gender end) AS male,
COUNT (case when gender = 'F' then gender end) AS female 
FROM
customer_data
WHERE lower(status) = 'attrited customer'
GROUP by 1
ORDER by age_range;"""
pd.read_sql(query3, con=engine)

,age_range,male,female
0,20-30,15,17
1,30-40,138,172
2,40-50,330,449
3,50-60,185,263
4,60++,29,29


In [50]:
query4 = """ 
SELECT income_category, COUNT(income_category) 
total_customer from 
customer_data
WHERE lower(status) = 'attrited customer'
GROUP by 1
ORDER by total_customer desc"""
pd.read_sql(query4, con=engine)

,income_category,total_customer
0,Less than $40K,612
1,$40K - $60K,271
2,$80K - $120K,242
3,$60K - $80K,189
4,Unknown,187
5,$120K +,126


## Visualisasi 
Pada project ni visualisasi data dilakukan menggunakan Tableau Public, yang dapat dilihat pada file <a href='Task_5_Ardi_Muhammad.pdf'> Task_5_Ardi_Muhammad.pdf